In [ ]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install faiss-gpu
!pip install einops
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm
!pip install rouge_score nltk bert_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/DATASCI266/project')

In [ ]:
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel

In [ ]:
import os
import re
import torch
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers import pipeline, BitsAndBytesConfig

In [ ]:
summary_evaluator = SummaryEvaluator()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load the XSum dataset
data_manager = DatasetManager(sample_size=200)
dataset = data_manager.load_sampled_dataset(dataset_label="test")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
%%capture

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         llm_int4_enable_fp32_cpu_offload=True)


llm_mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

llm_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Unused kwargs: ['llm_int4_enable_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
mistral_pipe = pipeline(
    "text-generation",
    model=llm_mistral_model,
    tokenizer=llm_mistral_tokenizer,
    max_length=1000,
    temperature=0.6,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2
)
mistral_pipe.model.config.pad_token_id = mistral_pipe.model.config.eos_token_id


In [ ]:
mistral_llm_lc = HuggingFacePipeline(pipeline=mistral_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    rag_template = """[INST] Give me a short one-sentence summarization with the following context: {context}[/INST]"""

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | mistral_llm_lc
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k):
    results = []
    i = 1
    for example in tqdm(dataset):
        print(i)
        text = example['document']
        sentences = re.split(r'[.?!]\s+', text)
        example_query = sentences[-1]
        summary_w_prompt = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        summary = summary_w_prompt.split("[/INST] ",1)[1]
        results.append(summary)
        i += 1
    return results

In [ ]:
references = [example['summary'] for example in dataset]

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:02<09:51,  2.97s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:07<12:09,  3.68s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [16:28<00:11,  5.92s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [16:31<00:05,  5.05s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [16:33<00:00,  4.97s/it]


In [ ]:
predictions1[0]

'Ms. Johnson realized how fortunate she was after experiencing a crash and expressed gratitude for being alive.'

In [ ]:
print("chunk_size=200, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.16355340421017103, 'rouge2': 0.022727393028021247, 'rougeL': 0.11805161372690731}
Average BLEU score: 0.01569592047558208


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4904729127883911
Average VECTOR_SIMILARITY score: 0.4022108316421509
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.17148312518924846, 'rouge2': 0.02634088875498529, 'rougeL': 0.1241629423885119}
Average BLEU score: 0.017847994981922893


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4894005060195923
Average VECTOR_SIMILARITY score: 0.3956124782562256


{'rouge': {'rouge1': 0.17148312518924846,
  'rouge2': 0.02634088875498529,
  'rougeL': 0.1241629423885119},
 'bleu': 0.017847994981922893,
 'bertscore': 0.4894005060195923,
 'vector_similarity': 0.3956124782562256}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=100, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:02<08:48,  2.65s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:04<07:45,  2.35s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [14:54<00:12,  6.19s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [14:56<00:04,  4.79s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [14:58<00:00,  4.49s/it]


In [ ]:
print("chunk_size=100, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=100, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1481490580868946, 'rouge2': 0.016999083347608645, 'rougeL': 0.10653123202199401}
Average BLEU score: 0.01427857602422953


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.46401137113571167
Average VECTOR_SIMILARITY score: 0.30907782912254333
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.15621017874457746, 'rouge2': 0.02220346295832801, 'rougeL': 0.11191458400855683}
Average BLEU score: 0.01705023701662339


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47125113010406494
Average VECTOR_SIMILARITY score: 0.32577696442604065
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.15040650308326317, 'rouge2': 0.016608330404093475, 'rougeL': 0.10788136938204368}
Average BLEU score: 0.014190772655581664


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4706514775753021
Average VECTOR_SIMILARITY score: 0.34304484724998474
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.15757892292618528, 'rouge2': 0.022296786035536652, 'rougeL': 0.11547545938911402}
Average BLEU score: 0.01619310645759771


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47789138555526733
Average VECTOR_SIMILARITY score: 0.34750980138778687


{'rouge': {'rouge1': 0.15757892292618528,
  'rouge2': 0.022296786035536652,
  'rougeL': 0.11547545938911402},
 'bleu': 0.01619310645759771,
 'bertscore': 0.47789138555526733,
 'vector_similarity': 0.34750980138778687}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:02<07:56,  2.39s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:05<08:57,  2.71s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [15:49<00:16,  8.08s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [15:50<00:06,  6.10s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [15:53<00:00,  4.77s/it]


In [ ]:
print("chunk_size=200, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.16144731562945033, 'rouge2': 0.022588921468041066, 'rougeL': 0.11491239970588268}
Average BLEU score: 0.017458177461525476


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4830622971057892
Average VECTOR_SIMILARITY score: 0.3700641095638275
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.17074098645209498, 'rouge2': 0.02371666897817765, 'rougeL': 0.12113039756080096}
Average BLEU score: 0.01616861461368576


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4898807108402252
Average VECTOR_SIMILARITY score: 0.3900403380393982
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.1675751676822101, 'rouge2': 0.02694788123619856, 'rougeL': 0.12345277860079806}
Average BLEU score: 0.017771052766477023


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.48963409662246704
Average VECTOR_SIMILARITY score: 0.3909071981906891
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.16723816427376836, 'rouge2': 0.024217319961978942, 'rougeL': 0.12018236744808149}
Average BLEU score: 0.016476956546261752


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4892538785934448
Average VECTOR_SIMILARITY score: 0.38639017939567566


{'rouge': {'rouge1': 0.16723816427376836,
  'rouge2': 0.024217319961978942,
  'rougeL': 0.12018236744808149},
 'bleu': 0.016476956546261752,
 'bertscore': 0.4892538785934448,
 'vector_similarity': 0.38639017939567566}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=100, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:02<09:16,  2.80s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:05<09:10,  2.78s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [13:56<00:08,  4.07s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [13:58<00:03,  3.32s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [14:01<00:00,  4.21s/it]


In [ ]:
print("chunk_size=100, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=100, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.15801105139752397, 'rouge2': 0.01921506606583566, 'rougeL': 0.11344954164640403}
Average BLEU score: 0.01536595238014643


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4698186218738556
Average VECTOR_SIMILARITY score: 0.3299219310283661
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.15710506437775554, 'rouge2': 0.020368767205138125, 'rougeL': 0.11536147224035785}
Average BLEU score: 0.016233235609477936


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47430846095085144
Average VECTOR_SIMILARITY score: 0.3366575241088867
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.16608403258716112, 'rouge2': 0.021270216052077982, 'rougeL': 0.12001912454481063}
Average BLEU score: 0.016055847149079937


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4773094654083252
Average VECTOR_SIMILARITY score: 0.3562484681606293
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.1633923196576763, 'rouge2': 0.022996584606057358, 'rougeL': 0.1176845291417337}


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BLEU score: 0.016169998663598063
Average BERTSCORE score: 0.4791804254055023
Average VECTOR_SIMILARITY score: 0.35523176193237305


{'rouge': {'rouge1': 0.1633923196576763,
  'rouge2': 0.022996584606057358,
  'rougeL': 0.1176845291417337},
 'bleu': 0.016169998663598063,
 'bertscore': 0.4791804254055023,
 'vector_similarity': 0.35523176193237305}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=150, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:02<08:06,  2.44s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:06<10:40,  3.24s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [15:05<00:11,  5.73s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [15:06<00:04,  4.36s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [15:08<00:00,  4.54s/it]


In [ ]:
print("chunk_size=150, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=150, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1642892936684006, 'rouge2': 0.024553764434951365, 'rougeL': 0.12102867555139758}
Average BLEU score: 0.017084341026169705


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.47344130277633667
Average VECTOR_SIMILARITY score: 0.3431813716888428
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.1691366656128041, 'rouge2': 0.023253495174039244, 'rougeL': 0.12502398466197456}
Average BLEU score: 0.016646336044760745


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4818683862686157
Average VECTOR_SIMILARITY score: 0.37112951278686523
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.16997613015510363, 'rouge2': 0.021220113435243677, 'rougeL': 0.12105258865329051}
Average BLEU score: 0.01588507694759749


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4843544363975525
Average VECTOR_SIMILARITY score: 0.3831527531147003
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.16810636577032645, 'rouge2': 0.027133766273881914, 'rougeL': 0.12156195804409582}
Average BLEU score: 0.01754594965151532


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.4856303334236145
Average VECTOR_SIMILARITY score: 0.37885910272598267


{'rouge': {'rouge1': 0.16810636577032645,
  'rouge2': 0.027133766273881914,
  'rougeL': 0.12156195804409582},
 'bleu': 0.01754594965151532,
 'bertscore': 0.4856303334236145,
 'vector_similarity': 0.37885910272598267}